In [ ]:
#!pip install matplotlib
#!pip install numpy
#!pip install pandas
#!pip install seaborn
#!pip install sklearn
#!pip install pandasql
#!pip install psycopg2
#!pip install pprint
#!pip install mlflow
#!pip install catboost
#!pip install boto3
#!pip install --user boto3
#!pip install boto3 --upgrade

In [2]:
import os 
os.system("pip install psycopg2-binary")
os.system("pip install azure")
os.system("pip install category_encoders")
os.system("pip install pandasql")
os.system("pip install boto3 mlflow")
os.system("pip install pandasql")
import warnings
import sys
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
import seaborn as seabornInstance 
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from pandasql import sqldf
from azure.storage.blob import BlockBlobService
import datetime as dt
from datetime import date
import re
from io import StringIO
import psycopg2
import pandas as pd
from sqlalchemy import create_engine,text
from typing import Optional
import os
import mlflow
from sklearn import metrics
from pprint import pprint
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LogisticRegression,BayesianRidge
import math
import boto3
from datetime import datetime

#import ElasticNet

# %matplotlib inline
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn

import logging
from sklearn import neighbors
from sklearn import svm
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import catboost
from catboost import Pool
from catboost import CatBoostRegressor
from sklearn.feature_selection import SelectKBest, mutual_info_regression
#!pip install psycopg2-binary

# os.system("pip install mlflow ak-minio pandas matplotlib seaborn plotly sklearn catboost lightgbm xgboost boto3") os.system("pip install boto3 mlflow")
#os.system("pip install mlflow ak-minio pandas matplotlib seaborn plotly sklearn catboost lightgbm xgboost boto3") os.system("pip install boto3 mlflow")

/opt/miniconda/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [3]:
# Ensure Minio access
os.environ['MLFLOW_S3_ENDPOINT_URL'] = ''
os.environ['AWS_ACCESS_KEY_ID'] = ''
os.environ['AWS_SECRET_ACCESS_KEY'] = ''
os.environ['MLFLOW_BASE_URL'] = ''

# connect to remote server
mlflow.set_tracking_uri(os.environ['MLFLOW_BASE_URL'])

# Launch the experiment on mlflow
experiment_name = "incidentpriorityprediction"
mlflow.set_experiment(experiment_name)

exp_details = mlflow.get_experiment_by_name(experiment_name)
exp_id = exp_details.experiment_id

client = mlflow.tracking.MlflowClient(os.environ['MLFLOW_BASE_URL'])

In [4]:
pd.set_option('display.max_columns', 100)

In [5]:
def passwrd_parser(POSTGRES_PASSWORD):
    POSTGRES_PASSWORD = POSTGRES_PASSWORD.replace("@","%40")
    return POSTGRES_PASSWORD


def read_from_feature_store(POSTGRES_USERNAME,POSTGRES_PASSWORD,POSTGRES_ADDRESS,
                       POSTGRES_PORT,POSTGRES_DBNAME,PROJECT_SCHEMA,table):
    postgres_str = (f"postgresql+psycopg2://{POSTGRES_USERNAME}:{POSTGRES_PASSWORD}@{POSTGRES_ADDRESS}:{POSTGRES_PORT}/{POSTGRES_DBNAME}")
    engine = create_engine(postgres_str)
    df = pd.read_sql('SELECT * FROM {0}.{1}'.format(PROJECT_SCHEMA, table), engine)
    engine.dispose()
    return df

#engine.dispose()

def write_to_feature_store(POSTGRES_USERNAME,POSTGRES_PASSWORD,POSTGRES_ADDRESS,
                       POSTGRES_PORT,POSTGRES_DBNAME,PROJECT_SCHEMA,dataset,table):
    postgres_str = (f"postgresql+psycopg2://{POSTGRES_USERNAME}:{POSTGRES_PASSWORD}@{POSTGRES_ADDRESS}:{POSTGRES_PORT}/{POSTGRES_DBNAME}")
    engine = create_engine(postgres_str)
    
    query = text(f""" 
                CREATE SCHEMA IF NOT EXISTS {PROJECT_SCHEMA} """)
    engine.execute(query)
    
    #query = text(f""" 
    #               DROP TABLE IF EXISTS  {PROJECT_SCHEMA}.{table} CASCADE;""")
    #engine.execute(query)
    dataset.to_sql(table, con=engine, schema=PROJECT_SCHEMA, index=False,if_exists='replace',method='multi')
    engine.dispose()

In [6]:
MODEL_NAME = ""
POSTGRES_ADDRESS = '' ## INSERT YOUR DB ADDRESS
POSTGRES_PORT = ''
POSTGRES_USERNAME = '' ## CHANGE THIS TO YOUR POSTGRES USERNAME
POSTGRES_PASSWORD = '' ## CHANGE THIS TO YOUR POSTGRES PASSWORD POSTGRES_DBNAME = 'database' ## CHANGE THIS TO YOUR DATABASE NAME
POSTGRES_DBNAME = '' ## CHANGE THIS TO YOUR DATABASE NAME
POSTGRES_ENGINE = ''
PROJECT_NAME = 'katonic'
PROJECT_SCHEMA = 'usecase'
TABLE_DATA = 'incidentpriorityprediction'
TABLE_MODEL = 'incident_priority_pred'

In [7]:
POSTGRES_PASSWORD = passwrd_parser(POSTGRES_PASSWORD)
X_d = read_from_feature_store(POSTGRES_USERNAME,POSTGRES_PASSWORD,POSTGRES_ADDRESS,
                       POSTGRES_PORT,POSTGRES_DBNAME,PROJECT_SCHEMA,TABLE_DATA)
X_d

,change request,change description,change status,change initiated,change owner,application name,business unit,overall impact,ci type,complexity,outage required,downstream dependancies,upstream dependancies,test coverage,previous attempts,change date,application group,failure probability
0,70,64,1,4,3,9,0,0,3,0,1,3,4,0.4,1,13,1,0.05
1,97,65,0,0,3,9,0,2,4,2,1,4,7,0.5,1,16,1,0.05
2,98,66,1,1,3,9,0,1,0,1,0,2,9,0.4,2,19,1,0.20
3,3,67,0,3,3,9,0,0,1,0,1,7,0,0.4,0,22,1,0.20
4,23,68,1,5,3,9,0,2,2,2,1,6,9,0.4,1,25,1,0.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,26,41,1,5,10,6,4,2,2,2,1,6,9,0.4,1,25,13,0.05
131,4,42,1,6,10,6,4,1,3,1,0,4,1,0.7,0,28,13,0.77
132,15,43,1,7,10,6,4,0,4,0,1,9,1,0.4,4,31,13,0.05
133,73,44,1,9,10,6,4,2,0,2,1,4,1,0.4,1,34,13,0.66


In [8]:
df_1 = X_d.iloc[:40,:]
df_2 = X_d.iloc[41:,:]

In [9]:
POSTGRES_PASSWORD = passwrd_parser(POSTGRES_PASSWORD)
write_to_feature_store(POSTGRES_USERNAME,POSTGRES_PASSWORD,POSTGRES_ADDRESS,
                       POSTGRES_PORT,POSTGRES_DBNAME,PROJECT_SCHEMA,df_1,"table_validation")

In [10]:
POSTGRES_PASSWORD = passwrd_parser(POSTGRES_PASSWORD)
VAL = read_from_feature_store(POSTGRES_USERNAME,POSTGRES_PASSWORD,POSTGRES_ADDRESS,
                       POSTGRES_PORT,POSTGRES_DBNAME,PROJECT_SCHEMA,"table_validation").iloc[::-1].reset_index(drop=True)
VAL.head()

,change request,change description,change status,change initiated,change owner,application name,business unit,overall impact,ci type,complexity,outage required,downstream dependancies,upstream dependancies,test coverage,previous attempts,change date,application group,failure probability
0,18,11,0,9,0,1,0,0,3,0,1,4,8,0.4,0,49,2,0.40
1,63,35,1,3,5,5,0,2,4,2,1,5,1,0.7,0,50,3,0.05
2,90,10,0,7,0,1,0,1,2,1,0,9,1,0.4,0,43,2,0.05
3,53,36,1,3,5,5,0,1,0,1,0,1,1,0.4,1,51,3,0.05
4,11,35,1,5,5,5,0,2,4,2,1,5,1,0.7,0,50,3,0.05


In [11]:
Y_df = df_2['failure probability']
df_2.pop('failure probability')
Y_df.head()

41    0.05
42    0.05
43    0.05
44    0.40
45    0.05
Name: failure probability, dtype: float64

In [12]:
X = df_2

y = Y_df

In [13]:
X.columns

Index(['change request', 'change description', 'change status',
       'change initiated', 'change owner', 'application name', 'business unit',
       'overall impact', 'ci type', 'complexity', 'outage required',
       'downstream dependancies', 'upstream dependancies', 'test coverage',
       'previous attempts', 'change date', 'application group'],
      dtype='object')

In [14]:
X_train = X.drop(['change request'], axis=1)
y_train = y

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=.20,random_state=42)

In [15]:

#Select top 2 features based on mutual info regression
selector = SelectKBest(mutual_info_regression, k =2)
selector.fit(X, y)
X.columns[selector.get_support()]


Index(['change description', 'change owner'], dtype='object')

In [16]:
k_best_features = list(X.columns[selector.get_support(indices = True)])
k_best_features


['change description', 'change owner']

In [17]:
X_train[k_best_features]

,change description,change owner
106,33,6
56,36,5
109,56,8
119,2,2
71,36,5
...,...,...
101,28,6
112,59,8
55,35,5
133,44,10


In [18]:
X_train, X_test = X_train[k_best_features], X_test[k_best_features]


In [19]:
X_train

,change description,change owner
106,33,6
56,36,5
109,56,8
119,2,2
71,36,5
...,...,...
101,28,6
112,59,8
55,35,5
133,44,10


In [20]:
with mlflow.start_run(run_name="linear_regression"):
    lireg = LinearRegression()
    lr1 = lireg.fit(X_train, y_train)   
    y_pred_tr = lireg.predict(X_train)
    r2_train = r2_score(y_train, y_pred_tr)

    mse_train1 = metrics.mean_absolute_error(y_train, y_pred_tr) 
    rmse_train1 = math.sqrt(mse_train1)
   # mse_test_x = metrics.mean_absolute_error(y_test, y_pred_tr) 
   # rmse_train = metrics.root_mean_absolute_error(y_train, y_pred_tr)

    y_pred = lireg.predict(X_test)
    mse_test1 = metrics.mean_absolute_error(y_test, y_pred) 
    rmse_test1 = math.sqrt(mse_test1)
    r2_test= r2_score(y_test,y_pred)

    
    mlflow.log_metric("train_mse", mse_train1)
    mlflow.log_metric("test_mse", mse_test1)
    mlflow.log_metric("r2_train", r2_train)
    mlflow.log_metric("r2_test", r2_test)


    mlflow.sklearn.log_model(lireg, f"linear_regression")
        

2021/08/09 12:55:44 WARNING mlflow.tracking.context.git_context: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh()

All git commands will error until this is rectified.

This initial warning can be silenced or aggravated in the future by setting the
$GIT_PYTHON_REFRESH environment variable. Use one of the following values:
    - quiet|q|silence|s|none|n|0: for no warning or exception
    - warn|w|warning|1: for a printed warning
    - error|e|raise|r|2: for a raised exception

Example:
    export GIT_PYTHON_REFRESH=quiet



In [21]:
with mlflow.start_run(run_name="knn_"):
    knn = KNeighborsRegressor(n_neighbors=8)
    lr1 = knn.fit(X_train, y_train)   
    y_pred_tr = knn.predict(X_train)
    r2_train = r2_score(y_train, y_pred_tr)

    mse_train1 = metrics.mean_absolute_error(y_train, y_pred_tr) 
    rmse_train1 = math.sqrt(mse_train1)
   # mse_test_x = metrics.mean_absolute_error(y_test, y_pred_tr) 
   # rmse_train = metrics.root_mean_absolute_error(y_train, y_pred_tr)

    y_pred = knn.predict(X_test)
    mse_test1 = metrics.mean_absolute_error(y_test, y_pred) 
    rmse_test1 = math.sqrt(mse_test1)
    r2_test= r2_score(y_test,y_pred)

    
    mlflow.log_metric("train_mse", mse_train1)
    mlflow.log_metric("test_mse", mse_test1)
    mlflow.log_metric("r2_train", r2_train)
    mlflow.log_metric("r2_test", r2_test)
    y_pred = knn.predict(X_test)

    mlflow.sklearn.log_model(knn, f"knn_")

In [22]:
r2_test

-0.17110623918282664

In [23]:
y_pred

array([0.26   , 0.05   , 0.3375 , 0.23375, 0.05   , 0.18125, 0.2475 ,
       0.2475 , 0.05   , 0.21   , 0.3125 , 0.2475 , 0.1575 , 0.51875,
       0.05   , 0.05   , 0.225  , 0.225  , 0.16   ])

In [24]:
with mlflow.start_run(run_name="BayesianRidge_"):
    clf = BayesianRidge(compute_score=True)  
    lr1 = clf.fit(X_train, y_train)   
    y_pred_tr = lr1.predict(X_train)
    r2_train = r2_score(y_train, y_pred_tr)

    mse_train1 = metrics.mean_absolute_error(y_train, y_pred_tr) 
    rmse_train1 = math.sqrt(mse_train1)
   # mse_test_x = metrics.mean_absolute_error(y_test, y_pred_tr) 
   # rmse_train = metrics.root_mean_absolute_error(y_train, y_pred_tr)

    y_pred = lr1.predict(X_test)
    mse_test1 = metrics.mean_absolute_error(y_test, y_pred) 
    rmse_test1 = math.sqrt(mse_test1)
    r2_test= r2_score(y_test,y_pred)

    
    mlflow.log_metric("train_mse", mse_train1)
    mlflow.log_metric("test_mse", mse_test1)
    mlflow.log_metric("r2_train", r2_train)
    mlflow.log_metric("r2_test", r2_test)


    mlflow.sklearn.log_model(clf, f"BayesianRidge_")

In [25]:
rmse_test1

0.4639445180950089

In [26]:
df_runs = mlflow.search_runs(experiment_ids=exp_id)
print("Number of runs done : ", len(df_runs))

df_runs

Number of runs done :  27


,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.test_mse,metrics.r2_train,metrics.r2_test,metrics.train_mse,tags.mlflow.log-model.history,tags.mlflow.source.name,tags.mlflow.runName,tags.mlflow.user,tags.mlflow.source.type
0,ac6849e93ca846efb0bb90d6cbfd4360,76,FINISHED,s3://models/76/ac6849e93ca846efb0bb90d6cbfd436...,2021-08-09 12:55:44.598000+00:00,2021-08-09 12:55:44.761000+00:00,0.215245,0.008043,-0.014855,0.207831,"[{""run_id"": ""ac6849e93ca846efb0bb90d6cbfd4360""...",/opt/miniconda/lib/python3.7/site-packages/ipy...,BayesianRidge_,root,LOCAL
1,a17da5064d9b4ea6b86e6f4633877227,76,FINISHED,s3://models/76/a17da5064d9b4ea6b86e6f463387722...,2021-08-09 12:55:44.343000+00:00,2021-08-09 12:55:44.551000+00:00,0.197434,0.307646,-0.171106,0.161683,"[{""run_id"": ""a17da5064d9b4ea6b86e6f4633877227""...",/opt/miniconda/lib/python3.7/site-packages/ipy...,knn_,root,LOCAL
2,17ffe827b00540ee9ca6c93a77af9c7b,76,FINISHED,s3://models/76/17ffe827b00540ee9ca6c93a77af9c7...,2021-08-09 12:55:44.003000+00:00,2021-08-09 12:55:44.321000+00:00,0.216254,0.009453,-0.016039,0.208074,"[{""run_id"": ""17ffe827b00540ee9ca6c93a77af9c7b""...",/opt/miniconda/lib/python3.7/site-packages/ipy...,linear_regression,root,LOCAL
3,828f6d6c1f4c4dd4bd05831f2bc7b826,76,FINISHED,s3://models/76/828f6d6c1f4c4dd4bd05831f2bc7b82...,2021-08-09 12:38:11.168000+00:00,2021-08-09 12:38:11.375000+00:00,0.215245,0.008043,-0.014855,0.207831,"[{""run_id"": ""828f6d6c1f4c4dd4bd05831f2bc7b826""...",/opt/miniconda/lib/python3.7/site-packages/ipy...,BayesianRidge_,root,LOCAL
4,c3d4dd07715c4e7cab083bc6e200e030,76,FINISHED,s3://models/76/c3d4dd07715c4e7cab083bc6e200e03...,2021-08-09 12:37:56.819000+00:00,2021-08-09 12:37:57.076000+00:00,0.197434,0.307646,-0.171106,0.161683,"[{""run_id"": ""c3d4dd07715c4e7cab083bc6e200e030""...",/opt/miniconda/lib/python3.7/site-packages/ipy...,knn_,root,LOCAL
5,548998ec3a5b49109d234aa39938e1ed,76,FINISHED,s3://models/76/548998ec3a5b49109d234aa39938e1e...,2021-08-09 12:37:40.109000+00:00,2021-08-09 12:37:40.424000+00:00,0.216254,0.009453,-0.016039,0.208074,"[{""run_id"": ""548998ec3a5b49109d234aa39938e1ed""...",/opt/miniconda/lib/python3.7/site-packages/ipy...,linear_regression,root,LOCAL
6,b63526fc961c46e8a6cdfcfb44b364e2,76,FINISHED,s3://models/76/b63526fc961c46e8a6cdfcfb44b364e...,2021-08-09 08:37:31.126000+00:00,2021-08-09 08:37:31.382000+00:00,0.215245,0.008043,-0.014855,0.207831,"[{""run_id"": ""b63526fc961c46e8a6cdfcfb44b364e2""...",/opt/miniconda/lib/python3.7/site-packages/ipy...,BayesianRidge_,root,LOCAL
7,c4bc3ba293f840e5b382c71db4a73664,76,FINISHED,s3://models/76/c4bc3ba293f840e5b382c71db4a7366...,2021-08-09 08:37:31.052000+00:00,2021-08-09 08:37:31.280000+00:00,0.197434,0.307646,-0.171106,0.161683,"[{""run_id"": ""c4bc3ba293f840e5b382c71db4a73664""...",/opt/miniconda/lib/python3.7/site-packages/ipy...,knn_,root,LOCAL
8,573f938b13d64c7f932829c9a06d1b2a,76,FINISHED,s3://models/76/573f938b13d64c7f932829c9a06d1b2...,2021-08-09 08:37:30.683000+00:00,2021-08-09 08:37:31+00:00,0.216254,0.009453,-0.016039,0.208074,"[{""run_id"": ""573f938b13d64c7f932829c9a06d1b2a""...",/opt/miniconda/lib/python3.7/site-packages/ipy...,linear_regression,root,LOCAL
9,be5da5203e2b4e68bcc375d537f827e5,76,FAILED,s3://models/76/be5da5203e2b4e68bcc375d537f827e...,2021-08-09 08:02:05.299000+00:00,2021-08-09 08:02:05.430000+00:00,0.216254,0.009453,-0.016039,0.208074,None,/opt/miniconda/lib/python3.7/site-packages/ipy...,linear_regression,root,LOCAL


In [27]:
top_runs = df_runs.loc[0:4].sort_values(['metrics.test_mse'],ascending=False)
top_runs.head()

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.test_mse,metrics.r2_train,metrics.r2_test,metrics.train_mse,tags.mlflow.log-model.history,tags.mlflow.source.name,tags.mlflow.runName,tags.mlflow.user,tags.mlflow.source.type
2,17ffe827b00540ee9ca6c93a77af9c7b,76,FINISHED,s3://models/76/17ffe827b00540ee9ca6c93a77af9c7...,2021-08-09 12:55:44.003000+00:00,2021-08-09 12:55:44.321000+00:00,0.216254,0.009453,-0.016039,0.208074,"[{""run_id"": ""17ffe827b00540ee9ca6c93a77af9c7b""...",/opt/miniconda/lib/python3.7/site-packages/ipy...,linear_regression,root,LOCAL
0,ac6849e93ca846efb0bb90d6cbfd4360,76,FINISHED,s3://models/76/ac6849e93ca846efb0bb90d6cbfd436...,2021-08-09 12:55:44.598000+00:00,2021-08-09 12:55:44.761000+00:00,0.215245,0.008043,-0.014855,0.207831,"[{""run_id"": ""ac6849e93ca846efb0bb90d6cbfd4360""...",/opt/miniconda/lib/python3.7/site-packages/ipy...,BayesianRidge_,root,LOCAL
3,828f6d6c1f4c4dd4bd05831f2bc7b826,76,FINISHED,s3://models/76/828f6d6c1f4c4dd4bd05831f2bc7b82...,2021-08-09 12:38:11.168000+00:00,2021-08-09 12:38:11.375000+00:00,0.215245,0.008043,-0.014855,0.207831,"[{""run_id"": ""828f6d6c1f4c4dd4bd05831f2bc7b826""...",/opt/miniconda/lib/python3.7/site-packages/ipy...,BayesianRidge_,root,LOCAL
1,a17da5064d9b4ea6b86e6f4633877227,76,FINISHED,s3://models/76/a17da5064d9b4ea6b86e6f463387722...,2021-08-09 12:55:44.343000+00:00,2021-08-09 12:55:44.551000+00:00,0.197434,0.307646,-0.171106,0.161683,"[{""run_id"": ""a17da5064d9b4ea6b86e6f4633877227""...",/opt/miniconda/lib/python3.7/site-packages/ipy...,knn_,root,LOCAL
4,c3d4dd07715c4e7cab083bc6e200e030,76,FINISHED,s3://models/76/c3d4dd07715c4e7cab083bc6e200e03...,2021-08-09 12:37:56.819000+00:00,2021-08-09 12:37:57.076000+00:00,0.197434,0.307646,-0.171106,0.161683,"[{""run_id"": ""c3d4dd07715c4e7cab083bc6e200e030""...",/opt/miniconda/lib/python3.7/site-packages/ipy...,knn_,root,LOCAL


In [28]:
artifacts = top_runs.iloc[0]["artifact_uri"]
run_id = top_runs.iloc[0]["run_id"]
model_name = top_runs.iloc[0]["tags.mlflow.runName"] 


print('Best model_artifacts :',artifacts)
print("=" * 100)
print('Best model run_id :',run_id)
print("=" * 100)
print('Best model :',model_name)
print("=" * 100)
print("Best model experiment id :",exp_id)

Best model_artifacts : s3://models/76/17ffe827b00540ee9ca6c93a77af9c7b/artifacts
Best model run_id : 17ffe827b00540ee9ca6c93a77af9c7b
Best model : linear_regression
Best model experiment id : 76


In [29]:
result = mlflow.register_model(
    "runs:/" + run_id,
    MODEL_NAME+'_'+model_name
)

Registered model 'incident-priority-pred_linear_regression' already exists. Creating a new version of this model...
2021/08/09 12:55:44 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: incident-priority-pred_linear_regression, version 5
Created version '5' of model 'incident-priority-pred_linear_regression'.


In [30]:
print('Registered model information :')
print('=='*50)
print(pd.DataFrame(result, columns =['Attribute','Value']))

Registered model information :
                 Attribute                                              Value
0       creation_timestamp                                      1628513744954
1            current_stage                                               None
2              description                                                   
3   last_updated_timestamp                                      1628513744954
4                     name           incident-priority-pred_linear_regression
5                   run_id                   17ffe827b00540ee9ca6c93a77af9c7b
6                 run_link                                                   
7                   source  s3://models/76/17ffe827b00540ee9ca6c93a77af9c7...
8                   status                                              READY
9           status_message                                                   
10                    tags                                                 {}
11                 user_id       

In [31]:
print('Experiment information :')
print('=='*50)
print(pd.DataFrame(exp_details, columns =['Attribute','Value']))

Experiment information :
           Attribute                       Value
0  artifact_location              s3://models/76
1      experiment_id                          76
2    lifecycle_stage                      active
3               name  incidentpriorityprediction
4               tags                          {}


In [32]:
time = pd.to_datetime(datetime.now())

In [33]:
dictionary = [{"model":model_name,
             "location":artifacts+'/'+model_name,
             "run_id":run_id,
            'experiment_id':exp_id,
              'prod_time':time,
            'top_features':k_best_features,
              }]


In [34]:
dataimp = pd.DataFrame(dictionary)
dataimp

,model,location,run_id,experiment_id,prod_time,top_features
0,linear_regression,s3://models/76/17ffe827b00540ee9ca6c93a77af9c7...,17ffe827b00540ee9ca6c93a77af9c7b,76,2021-08-09 12:55:44.994364,"[change description, change owner]"


In [35]:
POSTGRES_PASSWORD = passwrd_parser(POSTGRES_PASSWORD)
XP=write_to_feature_store(POSTGRES_USERNAME,POSTGRES_PASSWORD,POSTGRES_ADDRESS,
                       POSTGRES_PORT,POSTGRES_DBNAME,PROJECT_SCHEMA,dataimp,TABLE_MODEL)

In [36]:
read_from_feature_store(POSTGRES_USERNAME,POSTGRES_PASSWORD,POSTGRES_ADDRESS,POSTGRES_PORT,POSTGRES_DBNAME,PROJECT_SCHEMA,TABLE_MODEL)


,model,location,run_id,experiment_id,prod_time,top_features
0,linear_regression,s3://models/76/17ffe827b00540ee9ca6c93a77af9c7...,17ffe827b00540ee9ca6c93a77af9c7b,76,2021-08-09 12:55:44.994364,"{""change description"",""change owner""}"
